In [1]:
import tensorflow as tf
from utils import *
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Prepare VGG16 with dropout layers
raw_model = tf.keras.applications.vgg16.VGG16(include_top=True, weights='imagenet')
export_dropout_vgg16(0.8)
dropout_model = tf.keras.models.load_model("dropout_model.h5")
layer_name = "block5_conv3"

In [2]:
# Load and preprocess a sample image
input_image = np.load("images/ILSVRC2012_val_00034065.npy")
print("input shape:", input_image.shape)
preprocessed_input = np.expand_dims(tf.keras.applications.vgg16.preprocess_input(input_image), 0)

input shape: (224, 224, 3)


In [3]:
# Prediction of the raw model
category_index = np.argmax(raw_model(preprocessed_input))

In [4]:
# Raw grad-cam and score-cam
gradcam_image, _ = GradCam(raw_model,
        image = preprocessed_input,
        category_index = category_index,
       layer_name = layer_name,
        raw_array = input_image,
        dimension = 224
       )
scorecam_image, _ = ScoreCam(raw_model,
        image = preprocessed_input,
        category_index = category_index,
       layer_name = layer_name,
        raw_array = input_image,
        dimension = 224
       )

In [5]:
# MC-dropout grad-cam and score-cam
dropout_gradcam_image, _ = GradCam_Dropout(dropout_model,
        image = preprocessed_input,
        category_index = category_index,
       layer_name = layer_name,
        raw_array = input_image,
        dimension = 224,
        sample = 100
       )

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:02<00:00,  1.83s/it]


In [7]:
dropout_scorecam_image, _ = ScoreCam_Dropout(dropout_model,
        image = preprocessed_input,
        category_index = category_index,
       layer_name = layer_name,
        raw_array = input_image,
        dimension = 224,
        sample = 100
       )

In [ ]:
fignum = 4
fig = plt.figure()

ax0 = fig.add_subplot(1, fignum, 1)
ax0.imshow(Image.fromarray(gradcam_image))
ax0.set_title('Grad-CAM', fontsize=30)

ax1 = fig.add_subplot(1, fignum, 2)
ax1.imshow(Image.fromarray(dropout_gradcam_image))
ax1.set_title('weighted Grad-CAM', fontsize=30)

ax2 = fig.add_subplot(1, fignum, 3)
ax2.imshow(Image.fromarray(scorecam_image))
ax2.set_title('Score-CAM', fontsize=30)

ax3 = fig.add_subplot(1, fignum, 4)
ax3.imshow(Image.fromarray(scorecam_image))
ax3.set_title('weighted Score-CAM', fontsize=30)

fig.set_size_inches(15, 15)
fig.tight_layout()